In [1]:
%pwd

'c:\\Users\\sajit\\OneDrive\\Documents\\Desktop\\Pythonn\\Git\\Named-Entity-Recognition-News\\backend\\research'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\sajit\\OneDrive\\Documents\\Desktop\\Pythonn\\Git\\Named-Entity-Recognition-News\\backend'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EmbeddingPreparationConfig:
    root_dir: Path
    source_URL: str
    local_embedding_file: Path
    unzip_dir: Path
    params_embedding_dims: int
    params_vocab_size: int


In [5]:
from named_entity_recognition.constants import *
from named_entity_recognition.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_embedding_preparation_config(self) -> EmbeddingPreparationConfig:

        config = self.config.embedding_preparation

        create_directories([config.root_dir])

        embedding_preparation_config = EmbeddingPreparationConfig(
            root_dir = Path(config.root_dir),
            source_URL=config.source_URL,
            local_embedding_file=Path(config.local_embedding_file),
            unzip_dir=Path(config.unzip_dir),
            params_embedding_dims=self.params.EMBEDDING_DIM,
            params_vocab_size=self.params.VOCAB_SIZE
        )

        return embedding_preparation_config  
        

In [7]:
import os
import pickle
import urllib.request as request
import zipfile
from pathlib import Path
from named_entity_recognition import logger
from named_entity_recognition.utils.common import get_size
import tensorflow as tf
import numpy as np
from named_entity_recognition.entity.config_entity import EmbeddingPreparationConfig

c:\Users\sajit\anaconda3\envs\tf(GPU)3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
class EmbeddingPreparation:

    def __init__(self, config: EmbeddingPreparationConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_embedding_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_embedding_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_embedding_file))}")  
    
    def unzip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_embedding_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

    def prepare_word_to_vec(self):
        word_to_vec = {}
        with open(os.path.join(self.config.unzip_dir, f'glove.6B.{self.config.params_embedding_dims}d.txt'),'r', encoding='utf-8') as f:
            for line in f:
                values = line.split()
                word = values[0]
                vector = np.asarray(values[1:],'float32')
                word_to_vec[word]=vector
        self.word_to_vec = word_to_vec

    def prepare_tokenizer(self):
        tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token='<OOV>')
        tokenizer.fit_on_texts(self.word_to_vec.keys())
        self.tokenizer = tokenizer
        with open(os.path.join(self.config.root_dir, 'tokenizer.pickle'), 'wb') as f:
            pickle.dump(self.tokenizer, f)
    
    def prepare_embedding_weights(self):
        embedding_weights = np.zeros((self.config.params_vocab_size, self.config.params_embedding_dims))
        for word, i in self.tokenizer.word_index.items():
            if word in self.word_to_vec:
                embedding_weights[i] = self.word_to_vec[word]
        with open(os.path.join(self.config.root_dir, 'embedding.pickle'), 'wb') as f:
            pickle.dump(embedding_weights, f)

In [ ]:
try:
    config = ConfigurationManager()
    embedding_preparation_config = config.get_embedding_preparation_config()
    embedding_preparation = EmbeddingPreparation(embedding_preparation_config)
    embedding_preparation.download_file()
    embedding_preparation.unzip_file()
    embedding_preparation.prepare_word_to_vec()
    embedding_preparation.prepare_tokenizer()
    embedding_preparation.prepare_embedding_weights()
except Exception as e:
    raise e